# 19. AMES HOUSING: REVISITING OUTLIERS
---

## 1. Introducing the Data

In [1]:
import pandas as pd
import numpy as np
pd.set_option("display.max_columns", 99)
pd.set_option("display.max_rows", 999)
pd.set_option('precision', 3)

ames = pd.read_csv('data/Ames_Housing2_train14')
holdout = pd.read_csv('data/Ames_Housing2_test14')
print(ames.shape)
print(holdout.shape)
ames.head()

(2344, 246)
(586, 209)


,Lot Frontage,Lot Area,Lot Shape,Land Slope,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Mas Vnr Area,Exter Qual,Exter Cond,Bsmt Qual,Bsmt Cond,Bsmt Exposure,BsmtFin Type 1,BsmtFin SF 1,BsmtFin Type 2,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,Heating QC,Central Air,1st Flr SF,2nd Flr SF,Low Qual Fin SF,Gr Liv Area,Bsmt Full Bath,Bsmt Half Bath,Full Bath,Half Bath,Bedroom AbvGr,Kitchen AbvGr,Kitchen Qual,TotRms AbvGrd,Functional,Fireplaces,Fireplace Qu,Garage Yr Blt,Garage Finish,Garage Cars,Garage Area,Garage Qual,Garage Cond,Paved Drive,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,...,Exterior 2nd_Stone,Exterior 2nd_Stucco,Exterior 2nd_VinylSd,Exterior 2nd_Wd Sdng,Exterior 2nd_Wd Shng,Mas Vnr Type_BrkCmn,Mas Vnr Type_BrkFace,Mas Vnr Type_CBlock,Mas Vnr Type_None,Mas Vnr Type_Stone,Foundation_BrkTil,Foundation_CBlock,Foundation_PConc,Foundation_Slab,Foundation_Stone,Foundation_Wood,Heating_Floor,Heating_GasA,Heating_GasW,Heating_Grav,Heating_OthW,Heating_Wall,Electrical_FuseA,Electrical_FuseF,Electrical_FuseP,Electrical_Mix,Electrical_SBrkr,Garage Type_2Types,Garage Type_Attchd,Garage Type_Basment,Garage Type_BuiltIn,Garage Type_CarPort,Garage Type_Detchd,Garage Type_None,Misc Feature_Elev,Misc Feature_Gar2,Misc Feature_None,Misc Feature_Othr,Misc Feature_Shed,Sale Type_COD,Sale Type_CWD,Sale Type_Con,Sale Type_ConLD,Sale Type_ConLI,Sale Type_ConLw,Sale Type_New,Sale Type_Oth,Sale Type_VWD,Sale Type_WD
0,0.469,0.034,8,9,7,5,0.181,-0.381,0.527,5,5,7,5,2,2,-0.979,2,-0.294,1.235,0.098,5,10,0.559,-0.781,-0.098,-0.244,0,0,2,0,3,1,5,6,8,1,5,-0.021,6,2,0.047,5,5,9,-0.754,-0.696,-0.369,-0.099,-0.287,...,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1
1,0.410,2.307,4,3,5,5,-0.116,-0.814,-0.575,5,5,5,7,8,3,-0.428,6,4.191,-1.279,-0.078,5,10,0.013,1.481,-0.098,1.255,0,1,2,1,3,1,5,7,6,0,0,-0.372,9,2,0.325,5,5,9,3.139,-0.696,-0.369,-0.099,3.745,...,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1
2,0.014,-0.036,8,9,4,5,-0.280,-1.055,-0.575,5,5,0,0,0,0,-0.979,0,-0.294,-1.279,-2.399,5,10,0.180,-0.781,-0.098,-0.533,0,0,2,0,2,2,5,6,8,0,0,-0.568,3,2,-0.033,5,5,9,-0.754,-0.696,-0.369,-0.099,-0.287,...,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1
3,-0.441,-0.364,8,9,4,5,-0.709,-1.632,-0.575,5,5,0,0,0,0,-0.979,0,-0.294,-1.279,-2.399,3,5,-0.298,-0.781,-0.098,-0.898,0,0,2,0,2,2,5,6,8,0,0,-0.802,3,2,-0.231,5,5,9,-0.754,-0.696,-0.369,-0.099,-0.287,...,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1
4,-0.305,-0.311,8,9,4,6,-1.665,-1.632,-0.575,3,5,3,2,2,2,-0.979,2,-0.294,0.105,-1.024,7,5,-0.140,0.624,-0.098,0.413,0,0,2,0,4,2,3,8,8,0,0,-0.646,0,0,-2.209,0,0,3,-0.754,-0.696,1.839,-0.099,-0.287,...,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1


In [ ]:
X_train = ames.drop(['SalePrice'], axis=1)
y_train = 